In [ ]:
import os

import msgspec

from cffi import FFI

from extension_types import MatchTableType, SimpleMatchType, MatchTable, SimpleWord

In [ ]:
absolute_path = os.path.dirname("")

ffi = FFI()
ffi.cdef(
    open(os.path.join(absolute_path, "./matcher_c.h"), "r", encoding="utf-8").read()
)

lib = ffi.dlopen(os.path.join(absolute_path, "./matcher_c.so"))

In [ ]:
matcher = lib.init_matcher(
    msgspec.msgpack.encode({
        "test": [
            MatchTable(
                table_id=1,
                match_table_type=MatchTableType.Simple,
                simple_match_type=SimpleMatchType.MatchFanjian
                | SimpleMatchType.MatchDeleteNormalize,
                wordlist=["蔔", "你好"],
                exemption_simple_match_type=SimpleMatchType.MatchFanjian
                | SimpleMatchType.MatchDeleteNormalize,
                exemption_wordlist=[],
            )
        ]
    })
)

In [ ]:
lib.matcher_is_match(matcher, "你好".encode("utf-8"))

In [ ]:
res = lib.matcher_word_match(matcher, "你好".encode("utf-8"))
print(ffi.string(res).decode("utf-8"))
lib.drop_string(res)

In [ ]:
lib.drop_matcher(matcher)

In [ ]:
simple_matcher = lib.init_simple_matcher(
    msgspec.msgpack.encode(({
        SimpleMatchType.MatchFanjian | SimpleMatchType.MatchDeleteNormalize: [
            SimpleWord(1, "你好")
        ]
    }))
)

In [ ]:
lib.simple_matcher_is_match(simple_matcher, "你好".encode("utf-8"))

In [ ]:
res = lib.simple_matcher_process(simple_matcher, b"\xe4\xbd\xa0\xe5\xa5\xbd")
print(ffi.string(res).decode("utf-8"))
lib.drop_string(res)

In [ ]:
lib.drop_simple_matcher(simple_matcher)